# Answering instructions
This notebook demonstrates how you can modify default instructions for question types and answer comments.

## Default question instructions
Each EDSL question type includes default instructions to the model about how to answer the question.
We can view these instructions by inspecting the user prompt for a question that we have created.

For example, here we see that the default instruction for multiple choice questions is 
"*Only 1 option may be selected. Respond only with a string corresponding to one of the options. After the answer, you can put a comment explaining why you chose that option on the next line.*" 
This text is automatically appended to the question text:

In [1]:
from edsl import QuestionMultipleChoice, Survey

q = QuestionMultipleChoice(
    question_name = "primary_color",
    question_text = "What is the most common primary color?",
    question_options = ["Red", "Yellow", "Blue"]
)

Survey([q]).show_prompts()

,user_prompt,system_prompt,interview_index,question_name,scenario_index,agent_index,model,estimated_cost
0,"What is the most common primary color? Red Yellow Blue Only 1 option may be selected. Respond only with a string corresponding to one of the options. After the answer, you can put a comment explaining why you chose that option on the next line.",nan,0,primary_color,0,0,gpt-4o,0.000678


We can compare this with default instructions for other question types:

In [2]:
from edsl import QuestionCheckBox

Survey([QuestionCheckBox.example()]).show_prompts()

,user_prompt,system_prompt,interview_index,question_name,scenario_index,agent_index,model,estimated_cost
0,"Which of the following foods would you eat if you had to? 0: soggy meatpie 1: rare snails 2: mouldy bread 3: panda milk custard 4: McDonalds Minimum number of options that must be selected: 2. Maximum number of options that must be selected: 5. Please respond only with a comma-separated list of the code of the options that apply, with square brackets. E.g., [0, 1, 3]",nan,0,never_eat,0,0,gpt-4o,0.001012


In [3]:
from edsl import QuestionRank

Survey([QuestionRank.example()]).show_prompts()

,user_prompt,system_prompt,interview_index,question_name,scenario_index,agent_index,model,estimated_cost
0,"Rank your favorite foods. The options are: Pizza Pasta Salad Soup You can inlcude up to 2 options in your answer. Please respond only with a comma-separated list of the ranked options, with square brackets. E.g., ['Good', 'Bad', 'Ugly'] After the answer, you can put a comment explaining your choice on the next line.",nan,0,rank_foods,0,0,gpt-4o,0.000825


In [4]:
from edsl import QuestionLinearScale

Survey([QuestionLinearScale.example()]).show_prompts()

,user_prompt,system_prompt,interview_index,question_name,scenario_index,agent_index,model,estimated_cost
0,"How much do you like ice cream? 1 : I hate it 2 : 3 : 4 : 5 : I love it Only 1 option may be selected. Respond only with the code corresponding to one of the options. E.g., ""1"" or ""5"" by itself. After the answer, you can put a comment explaining why you chose that option on the next line.",nan,0,ice_cream,0,0,gpt-4o,0.000758


## Formatting answers & comments
We can see that each default instruction includes directions on (1) formatting the answer and (2) providing a comment about the answer.
When a question is administered, the contents of the comment that is returned are automatically stored in a separate field of the results.
We can see this when we run a question and inspect the columns of the results that have been created.
Here we run the multiple choice question created above:

In [5]:
from edsl import Model

m = Model("gemini-1.5-flash")

In [6]:
results = q.by(m).run()

We can see that the results include a `comment` field:

In [7]:
results.columns

,0
0,agent.agent_index
1,agent.agent_instruction
2,agent.agent_name
3,answer.primary_color
4,cache_used.primary_color_cache_used
5,comment.primary_color_comment
6,generated_tokens.primary_color_generated_tokens
7,iteration.iteration
8,model.maxOutputTokens
9,model.model


We can display it with any other fields:

In [8]:
results.select("primary_color", "primary_color_comment")

,answer.primary_color,comment.primary_color_comment
0,Red,"# There's no definitive scientific answer to this question, as the prevalence of a color as ""primary"" depends on the specific color model used (e.g., RYB, RGB). However, red is frequently cited as the most commonly used primary color in various contexts, including art and design."


## Turning off comments
If desired, we can omit the instruction to provide a comment by passing a parameter `include_comment=False` to the question constructor.
This may be desired if comments are not necessary or to save tokens.
Here we inspect how the question prompt has been modified and verify that the comment field in the results is blank:

In [9]:
from edsl import QuestionMultipleChoice, Survey

q = QuestionMultipleChoice(
    question_name = "primary_color",
    question_text = "What is the most common primary color?",
    question_options = ["Red", "Yellow", "Blue"],
    include_comment = False # optional
)

Survey([q]).show_prompts()

,user_prompt,system_prompt,interview_index,question_name,scenario_index,agent_index,model,estimated_cost
0,What is the most common primary color? Red Yellow Blue Only 1 option may be selected. Respond only with a string corresponding to one of the options.,nan,0,primary_color,0,0,gpt-4o,0.000438


In [10]:
from edsl import Model

m = Model("gemini-1.5-flash")

results = q.by(m).run()

results.select("primary_color", "primary_color_comment")

,answer.primary_color,comment.primary_color_comment
0,Red,nan


## Modifying comments
We can also modify the default instruction if we want to use the comment field in a different way.
This can be done by passing an optional parameter `answering_instruction` to the question constructor.
For example, here we pass an instruction that preserves the directions about the format of the answer 
to a multiple choice question ("*Respond only with a string corresponding to one of the options.*")
but replace the comments part of the instruction with a new instruction for the model to instead note
it's second choice answer. We include the original question in the survey as well for ease of comparison:

In [11]:
from edsl import QuestionMultipleChoice, Survey

q1 = QuestionMultipleChoice(
    question_name = "primary_color_v1",
    question_text = "What is the most common primary color?",
    question_options = ["Red", "Yellow", "Blue"]
)

q2 = QuestionMultipleChoice(
    question_name = "primary_color_v2",
    question_text = "What is the most common primary color?",
    question_options = ["Red", "Yellow", "Blue"],
    answering_instructions = """ 
    Respond only with a string corresponding to one of the options. 
    After the answer, please provide your second choice on the next line.
    """
)

survey = Survey([q1, q2])

survey.show_prompts()

,user_prompt,system_prompt,interview_index,question_name,scenario_index,agent_index,model,estimated_cost
0,"What is the most common primary color? Red Yellow Blue Only 1 option may be selected. Respond only with a string corresponding to one of the options. After the answer, you can put a comment explaining why you chose that option on the next line.",nan,0,primary_color_v1,0,0,gpt-4o,0.000678
1,"What is the most common primary color? Red Yellow Blue Only 1 option may be selected. Respond only with a string corresponding to one of the options. After the answer, please provide your second choice on the next line.",nan,0,primary_color_v2,0,0,gpt-4o,0.000640


In [12]:
from edsl import Model

m = Model("gemini-1.5-flash")

results = survey.by(m).run()

In [13]:
results.select("primary_color_v1", "primary_color_v1_comment", "primary_color_v2", "primary_color_v2_comment")

,answer.primary_color_v1,comment.primary_color_v1_comment,answer.primary_color_v2,comment.primary_color_v2_comment
0,Red,"# There's no definitive scientific answer to this question, as the prevalence of a color as ""primary"" depends on the specific color model used (e.g., RYB, RGB). However, red is frequently cited as the most commonly used primary color in various contexts, including art and design.",Red,Blue


## Further reading
Please see the [questions](https://docs.expectedparrot.com/en/latest/questions.html) page of the documentation for more examples and details on all of the required and optional parameters for question types!

Here we post this notebook to Coop for reference:

In [14]:
from edsl import Notebook

n = Notebook("answering_instructions_example.ipynb")
n.push(description = "Example answering instructions", visibility = "public")

{'description': 'Example answering instructions',
 'object_type': 'notebook',
 'url': 'https://www.expectedparrot.com/content/994ffea0-4597-4442-9ead-f58314a426d4',
 'uuid': '994ffea0-4597-4442-9ead-f58314a426d4',
 'version': '0.1.39.dev9',
 'visibility': 'public'}